# Cas pratique : Les données de marché

## Input

### Import des libraries

In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
import naas
import markdown2

### Récupération des données yahoo

In [2]:
# Lecture des données sur Yahoo finance
ticker = "^GSPC"
period1 = "1569172118"
period2 = "1600794518"
url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval=1d&events=history'
df = pd.read_csv(url)
df = df.sort_values(by="Date", ascending=True)
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-09-23,2983.500000,2999.149902,2982.229980,2991.780029,2991.780029,3186590000
1,2019-09-24,3002.429932,3007.979980,2957.729980,2966.600098,2966.600098,3868160000
2,2019-09-25,2968.350098,2989.820068,2952.860107,2984.870117,2984.870117,3318870000
3,2019-09-26,2985.729980,2987.280029,2963.709961,2977.620117,2977.620117,3077240000
4,2019-09-27,2985.469971,2987.310059,2945.530029,2961.790039,2961.790039,3243650000


### Variables

In [3]:
# Input
EXCEL_PATH = "Dashboard_Market_data.xlsx"
SHEET_TICKERS = 'TICKERS'
URL_YAHOO = "https://query1.finance.yahoo.com/v7/finance/download"
MARKDOWN_PATH = "FR_Market_data.md"

# Variabilisation des dates (bornes)
PERIOD_DEB = datetime(2020, 1, 1).strftime("%s")
PERIOD_FIN = datetime.now().strftime("%s")

# Output
OUTPUT_CSV = "BASE_MARKET-DATA.csv"

# Email
TODAY = datetime.now().strftime("%d/%m/%Y")
EMAIL_TO = ["jeremy.ravenel@cashstory.com"]
SUBJECT = f"📈 Votre brief quotidien: {TODAY}"

### Récupération de la liste des tickers

In [4]:
ref_tickers = pd.read_excel(EXCEL_PATH,
                            sheet_name=SHEET_TICKERS)
ref_tickers.tail(5)

,TYPOLOGY,TICKER,LABEL,ENTITIES
48,COMMODITIES,GC=F,COMEX Gold,GLOBAL
49,BONDS,^IRX,13 Week Treasury,GLOBAL
50,BONDS,^FVX,Yield 5 Years,GLOBAL
51,BONDS,^TNX,Yield 10 Years,GLOBAL
52,BONDS,^TYX,Yield 30 Years,GLOBAL


In [5]:
# Récupération de tous les tickers et création de la base de donnée tickers
tickers = ref_tickers['TICKER'].drop_duplicates()

# Initialisation du dataframe
data_tickers = pd.DataFrame()

for ticker in tickers:
    try:
        url = f'{URL_YAHOO}/{ticker}?period1={PERIOD_DEB}&period2={PERIOD_FIN}&interval=1d&events=history'
        df = pd.read_csv(url)
        df["TICKER"] = ticker
        data_tickers = pd.concat([data_tickers, df], axis=0)
        print(f"✅ Data successfully added for ticker {ticker}.")
    except:
        print(f"❌ Ticker {ticker} does not exist.")
data_tickers

✅ Data successfully added for ticker ^GSPC.
✅ Data successfully added for ticker ^DJI.
✅ Data successfully added for ticker ^IXIC.
✅ Data successfully added for ticker ^BVSP.
✅ Data successfully added for ticker ^GSPTSE.
❌ Ticker ^E3X.FGI does not exist.
✅ Data successfully added for ticker ^GDAXI.
✅ Data successfully added for ticker ^FCHI.
✅ Data successfully added for ticker ^FTSE.
❌ Ticker ^RTS.RS does not exist.
✅ Data successfully added for ticker ^AXJO.
✅ Data successfully added for ticker ^HSI.
✅ Data successfully added for ticker AMZN.
✅ Data successfully added for ticker EBAY.
✅ Data successfully added for ticker GOOGL.
✅ Data successfully added for ticker FB.
✅ Data successfully added for ticker BABA.
✅ Data successfully added for ticker BIDU.
✅ Data successfully added for ticker SAP.
✅ Data successfully added for ticker CRM.
✅ Data successfully added for ticker TWTR.
✅ Data successfully added for ticker PYPL.
✅ Data successfully added for ticker AAPL.
✅ Data successfully ad

,Date,Open,High,Low,Close,Adj Close,Volume,TICKER
0,2019-12-31,3215.179932,3231.719971,3212.030029,3230.780029,3230.780029,2.893810e+09,^GSPC
1,2020-01-02,3244.669922,3258.139893,3235.530029,3257.850098,3257.850098,3.458250e+09,^GSPC
2,2020-01-03,3226.360107,3246.149902,3222.340088,3234.850098,3234.850098,3.461290e+09,^GSPC
3,2020-01-06,3217.550049,3246.840088,3214.639893,3246.280029,3246.280029,3.674070e+09,^GSPC
4,2020-01-07,3241.860107,3244.909912,3232.429932,3237.179932,3237.179932,3.420380e+09,^GSPC
...,...,...,...,...,...,...,...,...
533,2021-10-01,2.060000,2.078000,2.045000,2.058000,2.058000,0.000000e+00,^TYX
534,2021-10-03,NaN,NaN,NaN,NaN,NaN,NaN,^TYX
535,2021-10-04,2.062000,2.081000,2.034000,2.047000,2.047000,0.000000e+00,^TYX
536,2021-10-05,2.051000,2.103000,2.051000,2.098000,2.098000,0.000000e+00,^TYX


## Model

### Nettoyage de la base tickers

In [6]:
# Initialisation du dataframe
data_clean = data_tickers.copy()

# Selection des colonnes à conserver
data_clean = data_clean[['Date', 'TICKER', 'Close']]

# Mise au format de la date
data_clean['Date'] = pd.to_datetime(data_clean['Date'], format='%Y-%m-%d')

# Suppression des valeurs nulles dans Close
data_clean = data_clean.dropna(subset=['Close'])

# Renomage de la colonne 'Close' en 'VALUE'
data_clean = data_clean.rename(columns={'Close': 'VALUE'})

# Format des colonnes en majuscule
data_clean.columns = data_clean.columns.str.upper()

# Affichage du dataframe
data_clean.head(5)

,DATE,TICKER,VALUE
0,2019-12-31,^GSPC,3230.780029
1,2020-01-02,^GSPC,3257.850098
2,2020-01-03,^GSPC,3234.850098
3,2020-01-06,^GSPC,3246.280029
4,2020-01-07,^GSPC,3237.179932


### Calcul de la valeur J-1

In [7]:
# Ajout des indexes dans le dataframe des valeurs
data_value = data_clean.copy()
data_value['INDEX_D-1'] = data_value.index - 1
data_value

,DATE,TICKER,VALUE,INDEX_D-1
0,2019-12-31,^GSPC,3230.780029,-1
1,2020-01-02,^GSPC,3257.850098,0
2,2020-01-03,^GSPC,3234.850098,1
3,2020-01-06,^GSPC,3246.280029,2
4,2020-01-07,^GSPC,3237.179932,3
...,...,...,...,...
532,2021-09-30,^TYX,2.092000,531
533,2021-10-01,^TYX,2.058000,532
535,2021-10-04,^TYX,2.047000,534
536,2021-10-05,^TYX,2.098000,535


In [8]:
# Création du dataframe pour récupérer la valeur précédente
data_value_last = data_clean.copy()
data_value_last['INDEX_D-1'] = data_value_last.index 
data_value_last = data_value_last.rename(columns={'VALUE': 'VALUE_D-1'})
data_value_last = data_value_last.drop(['DATE'], axis=1)
data_value_last

,TICKER,VALUE_D-1,INDEX_D-1
0,^GSPC,3230.780029,0
1,^GSPC,3257.850098,1
2,^GSPC,3234.850098,2
3,^GSPC,3246.280029,3
4,^GSPC,3237.179932,4
...,...,...,...
532,^TYX,2.092000,532
533,^TYX,2.058000,533
535,^TYX,2.047000,535
536,^TYX,2.098000,536


In [9]:
data_bdd = pd.merge(data_value,
                    data_value_last,
                    on=['TICKER', 'INDEX_D-1'],
                    how='left')

# Suppression de la colonne INDEX_D-1
data_bdd = data_bdd.drop(['INDEX_D-1'], axis=1)

# Affichage du dataframe
data_bdd.head(5)

,DATE,TICKER,VALUE,VALUE_D-1
0,2019-12-31,^GSPC,3230.780029,NaN
1,2020-01-02,^GSPC,3257.850098,3230.780029
2,2020-01-03,^GSPC,3234.850098,3257.850098
3,2020-01-06,^GSPC,3246.280029,3234.850098
4,2020-01-07,^GSPC,3237.179932,3246.280029


### Ajout de données complémentaires

In [10]:
# Initialisation du dataframe
data_enr = pd.DataFrame()

# Ajout des informations supplémentaires
data_enr = pd.merge(data_bdd, 
                    ref_tickers, 
                    on=['TICKER'],
                    how='left'
                   )

# Affichage du dataframe
data_enr.head(5)

,DATE,TICKER,VALUE,VALUE_D-1,TYPOLOGY,LABEL,ENTITIES
0,2019-12-31,^GSPC,3230.780029,NaN,INDICE,S&P 500,AMERICAS
1,2020-01-02,^GSPC,3257.850098,3230.780029,INDICE,S&P 500,AMERICAS
2,2020-01-03,^GSPC,3234.850098,3257.850098,INDICE,S&P 500,AMERICAS
3,2020-01-06,^GSPC,3246.280029,3234.850098,INDICE,S&P 500,AMERICAS
4,2020-01-07,^GSPC,3237.179932,3246.280029,INDICE,S&P 500,AMERICAS


### Creation des données globales

In [11]:
# Initialisation du dataframe
data_global = data_enr.copy()

# Liste des tickers à selectionner
tickers_list = ['^GSPC','^DJI','^FCHI','^FTSE','^HSI']

# Filtre des valeurs
data_global = data_global[data_global['TICKER'].isin(tickers_list)]

# Modification de la colonne ENTITIES
data_global['ENTITIES'] = 'GLOBAL'

# Concaténation des 2 dataframes
data_enr = pd.concat([data_enr, data_global], axis=0)

# Affichage du dataframe
data_enr.head(5)

,DATE,TICKER,VALUE,VALUE_D-1,TYPOLOGY,LABEL,ENTITIES
0,2019-12-31,^GSPC,3230.780029,NaN,INDICE,S&P 500,AMERICAS
1,2020-01-02,^GSPC,3257.850098,3230.780029,INDICE,S&P 500,AMERICAS
2,2020-01-03,^GSPC,3234.850098,3257.850098,INDICE,S&P 500,AMERICAS
3,2020-01-06,^GSPC,3246.280029,3234.850098,INDICE,S&P 500,AMERICAS
4,2020-01-07,^GSPC,3237.179932,3246.280029,INDICE,S&P 500,AMERICAS


### Calcul des variations, dernière date et market winner

In [12]:
data_final = data_enr.copy()

# Calcul des variations
data_final['VARV'] = data_final['VALUE'] - data_final['VALUE_D-1']
data_final['VARP'] = (data_final['VARV'] / abs(data_final['VALUE_D-1']))

# Tag sur la dernière date de chaque ticker
to_group = ['TICKER']
data_final['MAX_DATE'] = data_final.groupby(to_group)["DATE"].max()
data_final["DERNIERE_DATE"] = data_final.apply(lambda row: 1 if row['DATE'] == row['MAX_DATE'] else 0, axis=1)
data_final = data_final.drop("MAX_DATE", axis=1)

# Tag market winner / Loser
data_final.loc[data_final['VARP'] > 0, 'W_L'] = 1
data_final.loc[data_final['VARP'] <= 0, 'W_L'] = 0

# Affichage du dataframe
data_final.head(5)

,DATE,TICKER,VALUE,VALUE_D-1,TYPOLOGY,LABEL,ENTITIES,VARV,VARP,DERNIERE_DATE,W_L
0,2019-12-31,^GSPC,3230.780029,NaN,INDICE,S&P 500,AMERICAS,NaN,NaN,0,NaN
1,2020-01-02,^GSPC,3257.850098,3230.780029,INDICE,S&P 500,AMERICAS,27.070069,0.008379,0,1.0
2,2020-01-03,^GSPC,3234.850098,3257.850098,INDICE,S&P 500,AMERICAS,-23.000000,-0.007060,0,0.0
3,2020-01-06,^GSPC,3246.280029,3234.850098,INDICE,S&P 500,AMERICAS,11.429931,0.003533,0,1.0
4,2020-01-07,^GSPC,3237.179932,3246.280029,INDICE,S&P 500,AMERICAS,-9.100097,-0.002803,0,0.0


## Output

### Sauvegarde du fichier csv

In [13]:
# Ajout de la date de mise à jour
data_final['DATE_MAJ'] = datetime.now().strftime('%Y/%m/%d %H:%M:%S')

# Sauvegarde des données au format csv
data_final.to_csv(OUTPUT_CSV,
                  sep=";",
                  decimal=',',
                  index=False)

print('CSV sauvegardé dans le répertoire courant.')

CSV sauvegardé dans le répertoire courant.


### Exposition du csv

In [14]:
naas.asset.add(OUTPUT_CSV)

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


'https://public.naas.ai/amVyZW15LTJFcmF2ZW5lbC00MGNhc2hzdG9yeS0yRWNvbQ==/asset/935d31c82bbb87a43f4f780eda40573c8def13fdf2cdfff17c1dfc27a181'

### Exposition du rapport Excel

In [15]:
excel_url = naas.asset.add(EXCEL_PATH)

👌 Well done! Your Assets has been sent to production.



<IPython.core.display.Javascript object>

Button(button_style='primary', description='Copy URL', style=ButtonStyle())

Output()

PS: to remove the "Assets" feature, just replace .add by .delete


### Envoi de l'email executif

In [16]:
# Lecture du fichier html
content = open(MARKDOWN_PATH).read()
html = markdown2.markdown(content)
html = html.replace("{today}", TODAY)
html = html.replace("{excel}", excel_url)
print(html)

<p>Bonjour,</p>

<p>Voici ton brief quotidien sur la situation des marchés à la date du 06/10/2021.</p>

<p>Vous trouverez ci-joint <a href="https://public.naas.ai/ZWR1Y2F0aW9uLTQwY2FzaHN0b3J5LTJFY29t/asset/e80bac494cd9db68eb760b626930e9e90379f87444eeb71c53e94a08ca1d">votre rapport excel</a>.</p>

<p>Pour aller plus loin, consultez <a href="https://finance.yahoo.com/">Yahoo finance</a>.</p>

<p>La Team CashStory.</p>



In [17]:
# Fonction pour envoyer l'email
files = ["Dashboard_Market_data.xlsx"]
naas.notification.send(email_to=EMAIL_TO, subject=SUBJECT, html=html, files=files)

👌 💌 Email has been sent successfully !


### Automation

In [18]:
naas.dependency.add(MARKDOWN_PATH)
#naas.scheduler.add(cron="0 10 * * *")
#naas.scheduler.get_output()

👌 Well done! Your Dependency has been sent to production. 

PS: to remove the "Dependency" feature, just replace .add by .delete
